In [89]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import pathlib

from credentials import SAMPLE_DB, FULL_DB
from librarian import SAMPLE_DATA_DIR

import encoders
import db_connect
import helpers

In [90]:
DATA_DIR = SAMPLE_DATA_DIR
CHUNK_SIZE = 1024
MAIN_ACCURACY_METRIC = "mae"
GOOD_ACCURACY_DIRECTION = "min" # Some accuracy metrics go up for better results, some go down

ENCRYPTED_FILE_LIMIT = 0 # -1 to disable limit
SPLIT_SEED = 42
SPLIT_TEST_SIZE = 0.25     # 0.25 is default

LOAD_BEST_MODEL = False # If False, a new model will be created from scratch
SAVE_BEST_MODEL = True
TRAIN_MODEL = True
EPOCHS = 5

db = db_connect.DB(SAMPLE_DB)

In [91]:
# Get database IDs for encoders and key types

encoder_ids= {}
key_type_ids = {}

with db.get_session() as session:
    for encoder in encoders.ALL_ENCODER_NAMES:
        id = db.get_encoder_id(session, encoder)
        encoder_ids[encoder] = id

    print(f"Encoder IDs: {encoder_ids}")

    for key_type in encoders.KEY_NAMES:
        id = db.get_key_type_id(session, key_type)
        key_type_ids[key_type] = id

    print(f"Key Type IDs: {key_type_ids}")

Encoder IDs: {'None': 1, 'Simplifier': 2, 'Caesar Cipher': 3, 'Substitution Cipher': 4, 'Enigma Machine': 5}
Key Type IDs: {'Character Offset': 1, 'Character Map': 2, 'Rotor Settings': 3}


In [92]:
# Map source ID to plaintext file (1) details, and source ID to corresponding ciphertext files (1+) details
sid_to_p = {}
sid_to_c = {}

cipher_id = encoder_ids[encoders.ENCODER_CAESAR]
with db.get_session() as session:
    # Get all files encrypted with the cipher we care about
    encrypted_files = db.get_files_by_source_and_encoder(session, -1, cipher_id)

    if len(encrypted_files) > ENCRYPTED_FILE_LIMIT and ENCRYPTED_FILE_LIMIT > 0:
        encrypted_files = encrypted_files[0:ENCRYPTED_FILE_LIMIT]

    for c in encrypted_files:
        sid = c.source_id
    
        if sid not in sid_to_p:
            plaintext_ids = db.get_files_by_source_and_encoder(session, sid, encoder_ids[encoders.ENCODER_SIMPLIFIER])
            if len(plaintext_ids) != 1:
                raise Exception(f"Found {len(plaintext_ids)} plaintexts for source ID {sid}; should be exactly 1")
            sid_to_p[sid] = plaintext_ids[0]

        if sid not in sid_to_c:
            sid_to_c[sid] = []
        sid_to_c[sid].append(c)

len(sid_to_p), len(sid_to_c)

(6, 6)

In [93]:
# Build up the features (X, the cipher texts as values) and targets (y, the plain texts as values).
# Note the targets can be repeated since a plaintext can be encrypted repeatedly by different keys.
X = []
y = []

for sid in sid_to_p:
    plaintext = helpers.read_text_file(sid_to_p[sid].path)
    target_chunks = helpers.string_to_bytes(plaintext, CHUNK_SIZE)

    for c in sid_to_c[sid]:
        ciphertext = helpers.read_text_file(c.path)
        feature_chunks = helpers.string_to_bytes(ciphertext, CHUNK_SIZE)

        if len(target_chunks) != len(feature_chunks):
            raise Exception(f"Chunk count mismatch; {len(target_chunks)} != {len(feature_chunks)}")

        for i in range (len(target_chunks)):
            X.append(feature_chunks[i])
            y.append(target_chunks[i])

X = np.array(X)
y = np.array(y)

X.shape, y.shape, X[0].shape, y[0].shape

((7251, 1024), (7251, 1024), (1024,), (1024,))

In [94]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=SPLIT_TEST_SIZE, random_state=SPLIT_SEED)
print( len(X), len(y), len(X_train), len(X_test), len(y_train), len(y_test) )

7251 7251 5438 1813 5438 1813


In [95]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled.shape, X_test_scaled.shape

((5438, 1024), (1813, 1024))

In [96]:
if LOAD_BEST_MODEL:
    print(f"Loading model from {BEST_PATH}")
    nn = tf.keras.models.load_model(BEST_PATH)
else:
    print("Building new model")
    in_shape = (CHUNK_SIZE,)
    nn = tf.keras.models.Sequential()
    
    # Input layer
    nn.add(tf.keras.Input(shape=in_shape))
    
    # Hidden layers
    activations = ["tanh", "relu", "elu", "exponential", "gelu", "mish", "relu6", "tanh", "selu"]
    unit_counts = [CHUNK_SIZE*2, CHUNK_SIZE*2]
    for u in unit_counts:
        for a in activations:
            nn.add(tf.keras.layers.Dense(units=u, activation=a))
    
    # Output layer
    nn.add(tf.keras.layers.Dense(units=CHUNK_SIZE))
    
# Check the structure of the model
nn.summary()

Building new model


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_37 (Dense)                │ (None, 2048)           │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 2048)           │     4,196,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 2048)           │     4,196,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 2048)           │     4,196,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 2048)           │     4,196,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 2048)           │     4,196,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 2048)           │     4,196,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 2048)           │     4,196,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 2048)           │     4,196,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 2048)           │     4,196,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 2048)           │     4,196,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ (None, 2048)           │     4,196,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 2048)           │     4,196,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 2048)           │     4,196,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_51 (Dense)                │ (None, 2048)           │     4,196,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 2048)           │     4,196,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 2048)           │     4,196,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_54 (Dense)                │ (None, 2048)           │     4,196,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_55 (Dense)                │ (None, 1024)           │     2,098,176 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 75,535,360 (288.14 MB)

 Trainable params: 75,535,360 (288.14 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Set up training checkpoint to save after each epoch, if it is a new best model:
BEST_PATH = './best.keras'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=BEST_PATH,
    monitor=MAIN_ACCURACY_METRIC,
    mode=GOOD_ACCURACY_DIRECTION,
    save_best_only=True,
    save_weights_only=False,
    verbose=1)

if TRAIN_MODEL:
    print(f"Training model")
    
    # Compile the Sequential model together and customize metrics
    nn.compile(loss="mean_squared_error", optimizer="adamax", metrics=[MAIN_ACCURACY_METRIC])
    
    # Fit the model to the training data
    callbacks = None
    if SAVE_BEST_MODEL:
        callbacks = [model_checkpoint_callback]
    fit_model = nn.fit(X_train_scaled, y_train, epochs=EPOCHS, callbacks=callbacks)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Training model
Epoch 1/5


2024-11-19 21:16:40.120757: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_537', 132 bytes spill stores, 132 bytes spill loads

2024-11-19 21:16:40.447291: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_537', 16 bytes spill stores, 16 bytes spill loads

2024-11-19 21:16:40.447374: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1065', 48 bytes spill stores, 48 bytes spill loads

2024-11-19 21:16:40.647124: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1445', 36 bytes spill stores, 40 bytes spill loads

2024-11-19 21:16:40.702739: I external/local_xla/xla/stream_

169/170 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 1012.1085 - mae: 23.7886

2024-11-19 21:16:49.221475: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_565', 4 bytes spill stores, 4 bytes spill loads

2024-11-19 21:16:49.251978: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_523', 92 bytes spill stores, 92 bytes spill loads

2024-11-19 21:16:51.120073: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_565', 204 bytes spill stores, 204 bytes spill loads

2024-11-19 21:16:51.568590: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1065', 92 bytes spill stores, 92 bytes spill loads

2024-11-19 21:16:51.787266: I external/local_xla/xla/stream_exe

170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 1009.3297 - mae: 23.7515
Epoch 1: mae improved from inf to 17.48652, saving model to ./best.keras
170/170 ━━━━━━━━━━━━━━━━━━━━ 71s 374ms/step - loss: 1006.5835 - mae: 23.7148
Epoch 2/5
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 381.5742 - mae: 15.1795
Epoch 2: mae improved from 17.48652 to 15.17674, saving model to ./best.keras


In [ ]:
def decode_chunks_with_model(chunks: list[list], model, scaler, input_already_scaled = True) -> list[list]:
    if input_already_scaled:
        return model.predict(chunks)
    else:
        return model.predict(scaler.transform(chunks))

def decode_text_with_model(ciphertext: str, model, scaler) -> str:
    chunks = helpers.string_to_bytes(ciphertext, CHUNK_SIZE)
    decoded_chunks = decode_chunks_with_model(chunks, model, scaler, input_already_scaled = False)
    return helpers.bytes_to_string(decoded_chunks)

ciphertext_path = sid_to_c[list(sid_to_c.keys())[0]][0].path
ciphertext = helpers.read_text_file(ciphertext_path)
ciphertext = ciphertext[0:CHUNK_SIZE*2]
decode_text_with_model(ciphertext, nn, X_scaler)